# Análisis de datos de dataset SNBE al 2023_04

Hoy (2023-04-06) me enteré, [por un reel](https://www.instagram.com/reel/Cqn3HJ-O0wv/), que se consiguieron los datos del **Sistema Nacional de Billetaje Electrónico** (SNBE) asi que me puse en contacto con Belén y Marcelo para poder hecharles una mirada.

En el reel dice que son unos 8 CDs 🤪, y cuando obtuve acceso a los datos me encontré con un archio `.rar` partido en 8 (jeje). Al descomprimirlo aparecieron 15 archivos `.csv`.

## Cuantos archivos tenemos?

In [5]:
%%bash
find data/source/INFORMACION\ SNBE/ -type f  -exec md5sum {} \;

55874af68237ec2265283a2cf0b64fc9  data/source/INFORMACION SNBE/1- Enero_2022.csv
6c01bf153a45bd76a41cf7590a279d3e  data/source/INFORMACION SNBE/10- Octubre_2022.csv
b85f8ec08ea310f8e190083620ce9333  data/source/INFORMACION SNBE/11- Noviembre_2022.csv
a54cfae67cc3f6c3930366a66e1a9281  data/source/INFORMACION SNBE/12- Diciembre_2022.csv
5e527dad7b8066fec36db8b77766df54  data/source/INFORMACION SNBE/13- Enero_2023.csv
579bc8705138385d31f6160312e45580  data/source/INFORMACION SNBE/14- Febrero_2023.csv
edad9d2428cdf9ba866dbdda69a3298b  data/source/INFORMACION SNBE/15- Marzo_2023.csv
663cdbf31c2d49a0d328476e112a4f63  data/source/INFORMACION SNBE/2- Febrero_2022.csv
70bbb1ffac70eef02f0f0a8964305830  data/source/INFORMACION SNBE/3- Marzo_2022.csv
d90fed7b572614097bba2ebb569117af  data/source/INFORMACION SNBE/4- Abril_2022.csv
b1a327b74c54d85c0676c8609d47221e  data/source/INFORMACION SNBE/5- Mayo_2022.csv
46c98219fbd78bc1dcc0febb9d406c9e  data/source/INFORMACION SNBE/6- Junio_2022.csv
217533cf9

Un archivo por mes desde Enero del 2022 hasta Marzo del 2023. Supongo que está bien teniendo en cuenta que tienen que ser muchísimos registros, y como son `csv` son texto plano.

## Cuanto será que pesan los archivos?

In [2]:
%%bash
du -bsh data/source/INFORMACION\ SNBE/*

1.8G	data/source/INFORMACION SNBE/10- Octubre_2022.csv
1.8G	data/source/INFORMACION SNBE/11- Noviembre_2022.csv
1.8G	data/source/INFORMACION SNBE/12- Diciembre_2022.csv
1.6G	data/source/INFORMACION SNBE/13- Enero_2023.csv
1.5G	data/source/INFORMACION SNBE/14- Febrero_2023.csv
1.9G	data/source/INFORMACION SNBE/15- Marzo_2023.csv
1.3G	data/source/INFORMACION SNBE/1- Enero_2022.csv
1.4G	data/source/INFORMACION SNBE/2- Febrero_2022.csv
1.7G	data/source/INFORMACION SNBE/3- Marzo_2022.csv
1.6G	data/source/INFORMACION SNBE/4- Abril_2022.csv
1.7G	data/source/INFORMACION SNBE/5- Mayo_2022.csv
1.7G	data/source/INFORMACION SNBE/6- Junio_2022.csv
1.8G	data/source/INFORMACION SNBE/7- Julio_2022.csv
1.9G	data/source/INFORMACION SNBE/8- Agosto_2022.csv
1.8G	data/source/INFORMACION SNBE/9- Setiembre_2022.csv


## Cuantos registros tienen los archivos?

In [3]:
%%bash
wc -l data/source/INFORMACION\ SNBE/*

   15137196 data/source/INFORMACION SNBE/10- Octubre_2022.csv
   15421223 data/source/INFORMACION SNBE/11- Noviembre_2022.csv
   15264875 data/source/INFORMACION SNBE/12- Diciembre_2022.csv
   13464581 data/source/INFORMACION SNBE/13- Enero_2023.csv
   13076949 data/source/INFORMACION SNBE/14- Febrero_2023.csv
   16569684 data/source/INFORMACION SNBE/15- Marzo_2023.csv
   11372718 data/source/INFORMACION SNBE/1- Enero_2022.csv
   12349177 data/source/INFORMACION SNBE/2- Febrero_2022.csv
   14717119 data/source/INFORMACION SNBE/3- Marzo_2022.csv
   13361426 data/source/INFORMACION SNBE/4- Abril_2022.csv
   14645764 data/source/INFORMACION SNBE/5- Mayo_2022.csv
   14850336 data/source/INFORMACION SNBE/6- Junio_2022.csv
   15433715 data/source/INFORMACION SNBE/7- Julio_2022.csv
   16025858 data/source/INFORMACION SNBE/8- Agosto_2022.csv
   15789069 data/source/INFORMACION SNBE/9- Setiembre_2022.csv
  217479690 total


En fin, vamos a leerlos en un `DataFrame` de `spark` y guardarlos como parquet rápidamente.

## A ver un archivo?

In [10]:
%%bash
head -1 data/source/INFORMACION\ SNBE/1-\ Enero_2022.csv

serialtarjeta;idsam;fechahoraevento;producto;montoevento;consecutivoevento;identidad;tipoevento;latitude;longitude;idrutaestacion;tipotransporte


Al final no eran `csv` pero si un text delimited file separado por `;`.

Veo que tiene información sensible, la columna `serialtarjeta`. Es por eso que muestro solo la primera fila.

## Cargo los datos a Spark

El código en esta sección es el resultado de un proceso iterativo para asignar tipos de datos correctos a las columnas.

In [52]:
import glob

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()
snbe = spark.read.csv("data/source/INFORMACION SNBE/*.csv", sep=";", header=True)
snbe.printSchema()

snbe = (
    snbe
    .select(
        F.expr("serialtarjeta").alias("serial_tarjeta"),
        F.expr("idsam").alias("id_sam"),
        F.expr("to_timestamp(substring(fechahoraevento, 1, 23), 'yyyy/MM/dd HH:mm:ss.SSS')").alias("timestamp_evento"),
        "producto",
        F.expr("cast(montoevento as int)").alias("monto_evento"),
        F.expr("cast(consecutivoevento as int)").alias("consecutivo_evento"),
        F.expr("cast(identidad as int)").alias("identidad"),
        F.expr("cast(tipoevento as int)").alias("tipo_evento"),
        F.expr("cast(latitude as float)").alias("latitude"),
        F.expr("cast(longitude as float)").alias("longitude"),
        F.expr("idrutaestacion").alias("id_rutaestacion"),
        F.expr("tipotransporte").alias("tipo_transporte"),
    )
    .withColumn("event_year", F.expr("year(timestamp_evento)"))
    .withColumn("event_month", F.expr("month(timestamp_evento)"))
    .withColumn("event_day", F.expr("day(timestamp_evento)"))
    .withColumn("event_hour", F.expr("year(timestamp_evento)"))
    .withColumn("event_minute", F.expr("year(timestamp_evento)"))
)
snbe.printSchema()

root
 |-- serialtarjeta: string (nullable = true)
 |-- idsam: string (nullable = true)
 |-- fechahoraevento: string (nullable = true)
 |-- producto: string (nullable = true)
 |-- montoevento: string (nullable = true)
 |-- consecutivoevento: string (nullable = true)
 |-- identidad: string (nullable = true)
 |-- tipoevento: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- idrutaestacion: string (nullable = true)
 |-- tipotransporte: string (nullable = true)

root
 |-- serial_tarjeta: string (nullable = true)
 |-- id_sam: string (nullable = true)
 |-- timestamp_evento: timestamp (nullable = true)
 |-- producto: string (nullable = true)
 |-- monto_evento: integer (nullable = true)
 |-- consecutivo_evento: integer (nullable = true)
 |-- identidad: integer (nullable = true)
 |-- tipo_evento: integer (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- id_rutaestacion: string (nullab

In [55]:
(
    snbe.write
    .partitionBy("event_year","event_month", "event_day", "event_hour", "event_minute")
    .parquet("data/raw/snbe")
)